In [1]:
from matplotlib import pyplot as plt
from time import sleep
import itertools
import TimeTagger
import numpy as np
import asyncio
import plotly.graph_objs as go
from ipywidgets import Button


In [23]:
# create time tagger object (link to device) and get the active channels
tagger = TimeTagger.createTimeTagger()
input_channels = tagger.getChannelList(TimeTagger.ChannelEdge.Rising)
for ch in input_channels:
    tagger.setTriggerLevel(ch, 0.5)

# get every combination of 2 channels
groups = [(3, 4), (3, 1), (2, 4), (2, 1)]

# 30ns coincidence window to match qutools (coincidence window is in ps)
coincidenceWindowSI = 30e-9 # seconds
coincidences_vchannels = TimeTagger.Coincidences(tagger, groups, coincidenceWindow=coincidenceWindowSI * 1e12)
groups

[(3, 4), (3, 1), (2, 4), (2, 1)]

In [31]:
#trace = TimeTagger.Counter(tagger, [2], G10**9, 3000)
binwidthSI = 0.01
# convert binwidth to ps
traces = []
for vch in coincidences_vchannels.getChannels():
    counter = TimeTagger.Counter(tagger=tagger, channels=[vch], binwidth=binwidthSI * 1e12, n_values=1000)
    traces.append(counter)

# Init figure
fig_trace = go.FigureWidget()

# add scatter for each virtual channel
for trace, group in zip(traces, groups):
    fig_trace.add_scatter(x=trace.getIndex(), y=trace.getData()[0], name=f"|{'R' if group[0] == 3 else 'T'}, {'R' if group[1] == 4 else 'T'}>")

async def update_trace2():
    currentMax = 0
    while True:
        
        for i, trace in enumerate(traces):
            # get y_data and coinvert to Hz based on binwidth
            y_data = trace.getData()[0] / binwidthSI
            
            # update plotrange
            ymax = np.max(y_data)
            if ymax > currentMax:
                currentMax = 1.2 * ymax
                fig_trace.update_layout(yaxis=dict(range=[0, ymax]))

            fig_trace.data[i].y = y_data
        
        await asyncio.sleep(0.05)

async def update_trace():
    currentMax = 0
    while True:
        # Batch updates for all traces
        with fig_trace.batch_update():
            for i, trace in enumerate(traces):
                y_data = trace.getData()[0] / binwidthSI

                ymax = np.max(y_data)
                if ymax > currentMax:
                    currentMax = 1 * ymax
                    fig_trace.update_layout(yaxis=dict(range=[0, ymax]))

                fig_trace.data[i].y = y_data

        await asyncio.sleep(0.05)  # Adjust update interval as needed

try:
    task_trace.cancel()
except:
    pass
loop = asyncio.get_event_loop()
task_trace = loop.create_task(update_trace())

button_trace = Button(description="stop")
button_trace.on_click(lambda a: task_trace.cancel())

display(fig_trace, button_trace)

FigureWidget({
    'data': [{'name': '|R, R>',
              'type': 'scatter',
              'uid': '2180decc-15eb-4194-9322-e6d9319c90ca',
              'x': array([            0,   10000000000,   20000000000, ..., 9970000000000,
                          9980000000000, 9990000000000], dtype=int64),
              'y': array([0, 0, 0, ..., 0, 0, 0])},
             {'name': '|R, T>',
              'type': 'scatter',
              'uid': '36b2a6b2-a79a-41da-804a-cd8a460e5f16',
              'x': array([            0,   10000000000,   20000000000, ..., 9970000000000,
                          9980000000000, 9990000000000], dtype=int64),
              'y': array([0, 0, 0, ..., 0, 0, 0])},
             {'name': '|T, R>',
              'type': 'scatter',
              'uid': 'e7799033-2b3f-4765-b87d-78481a45aedd',
              'x': array([            0,   10000000000,   20000000000, ..., 9970000000000,
                          9980000000000, 9990000000000], dtype=int64),
              'y'

Button(description='stop', style=ButtonStyle())